In [132]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math 
from matplotlib.figure import Figure
pd.set_option('display.max_colwidth', None)
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [133]:
file=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')

In [134]:
details=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx')

In [135]:
details

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Data Description:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,ID,Customer ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Age,Customer's age in completed years,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Experience,#years of professional experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Income,Annual income of the customer ($000),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
y=file['Personal Loan']

In [138]:
X=file.drop('Personal Loan', axis='columns')

In [139]:
X

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,1,0


In [140]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

!pip install category_encoders
%matplotlib inline

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [142]:

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (4000, 13) (4000,)
Test (1000, 13) (1000,)


In [143]:
print(X_train)

        ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  \
868    869   40          15     161     94596       2    3.3          1   
936    937   62          32      19     92109       1    1.5          3   
4034  4035   35          11      82     94022       2    1.7          2   
1923  1924   45          19      22     90639       1    0.2          1   
3330  3331   34           9      32     95054       4    1.1          3   
...    ...  ...         ...     ...       ...     ...    ...        ...   
3335  3336   35          10     118     92069       2    7.8          1   
1099  1100   30           6      52     92717       3    0.7          2   
2514  2515   41          16      25     92182       2    0.1          2   
3606  3607   43          18       9     96145       2    0.0          3   
2575  2576   42          16      41     90401       3    0.5          3   

      Mortgage  Securities Account  CD Account  Online  CreditCard  
868          0                

In [144]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [145]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [146]:


model = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=1000, subsample=0.5)

In [147]:
model.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=1000, subsample=0.5)

In [148]:
predictions=model.predict(X_test)


In [149]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 3566 to 4121
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1000 non-null   int64  
 1   Age                 1000 non-null   int64  
 2   Experience          1000 non-null   int64  
 3   Income              1000 non-null   int64  
 4   ZIP Code            1000 non-null   int64  
 5   Family              1000 non-null   int64  
 6   CCAvg               1000 non-null   float64
 7   Education           1000 non-null   int64  
 8   Mortgage            1000 non-null   int64  
 9   Securities Account  1000 non-null   int64  
 10  CD Account          1000 non-null   int64  
 11  Online              1000 non-null   int64  
 12  CreditCard          1000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 109.4 KB


In [150]:
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99       909\n           1       0.95      0.91      0.93        91\n\n    accuracy                           0.99      1000\n   macro avg       0.97      0.95      0.96      1000\nweighted avg       0.99      0.99      0.99      1000\n'

In [151]:

confusion_matrix(y_test,predictions)

array([[905,   4],
       [  8,  83]])

In [152]:
accuracy_score(y_test,predictions)

0.988

In [153]:
print('The Precision Score- What proportion of positive identifications was actually correct?: ' + str(100*precision_score(y_test,predictions))+'%')
print('The Recall Score - What proportion of actual positives was identified correctly?: ' + str(100*recall_score(y_test,predictions))+'%')

The Precision Score- What proportion of positive identifications was actually correct?: 95.40229885057471%
The Recall Score - What proportion of actual positives was identified correctly?: 91.20879120879121%


In [154]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=15)
print('Cross-Validation Accuracy Scores', scores.mean())
recall = cross_val_score(model, X_train, y_train, cv=15, scoring='recall')
print('Recall', recall.mean())
precision = cross_val_score(model, X_train, y_train, cv=15, scoring='precision')
print('Precision', precision.mean())

Cross-Validation Accuracy Scores 0.9852505796326022
Recall 0.9022564102564105
Precision 0.9416289896362359


In [155]:
# example of grid searching key hyperparameters for GradientBoostingClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# define dataset
#X, y = make_blobs(n_samples=1000, centers=2, n_features=100, cluster_std=20)
# define models and parameters
model = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='recall',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.905780 using {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.7}
0.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.5}
0.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 0.7}
0.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10, 'subsample': 1.0}
0.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}
0.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
0.000000 (0.000000) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
0.803801 (0.061815) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.5}
0.807220 (0.058665) with: {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.7}
0.805511 (0.060828) with: {'learning_rate': 0.001, 'm

# Accuracy: 98.45%
# Precision 94.33%
# Recall: 90.48%


# 





# Predicting probability of personal loan per customer ID

In [120]:
#y_hats = model.predict(X_test)
#y_hats = model.predict(X)
y_hats = model.predict_proba(X)

y_hats = np.delete(y_hats, 0, axis=1)
y_hats
#.predict_proba(merged.values)
#y_hats_df = pd.DataFrame(data = y_hats, columns = ['y_hats'], index = X_test.index.copy())
y_hats_df = pd.DataFrame(data = y_hats, columns = ['Model Prediction Probability'], index = X.index.copy())
df_out1 = pd.merge(X, y, how = 'left', left_index = True, right_index = True)
df_out = pd.merge(df_out1, y_hats_df, how = 'left', left_index = True, right_index = True)

In [126]:
df_out=df_out.round({'Model Prediction Probability': 4})

In [127]:
df_out

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan,Model Prediction Probability
0,1,25,1,49,91107,4,1.6,1,0,1,0,0,0,0,0.0
1,2,45,19,34,90089,3,1.5,1,0,1,0,0,0,0,0.0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,0.0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,0.0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,1,0,0,0.0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,1,0,0,0.0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0,0.0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,1,0,0,0.0


In [128]:
df_out.to_excel('check_loan.xlsx')